In [1]:
import os
from glob import glob
from twelvelabs import TwelveLabs
from twelvelabs.models.task import Task

# The path to the directory containing the videos you wish to upload.
# VIDEO_PATH = "./videos/*.mp4" # Example: "/videos/*.mp4
VIDEO_PATH =r'/Users/jaredyu/Desktop/projects/twelve_labs/technical_assignment/twelve-labs-sample-proj/static/videos/AWS AppFabric introduction.mp4'
API_KEY = os.environ['API_KEY']
INDEX_NAME = "test_index-123213123123"

In [2]:
client = TwelveLabs(api_key=API_KEY)

index = client.index.create(
    name=INDEX_NAME,
    engines=[
        {
            "name": "pegasus1",
            "options": ["visual", "conversation"],
        }
    ]
)
print(f"Created index: id={index.id} name={index.name} engines={index.engines}")

Created index: id=6670f2c72f5f583ac97c8cf7 name=test_index-123213123123 engines=root=[Engine(name='pegasus1', options=['visual', 'conversation'], addons=None)]


In [3]:
video_files = glob(VIDEO_PATH)
for video_file in video_files:
  print(f"Uploading {video_file}")
  task = client.task.create(index_id=index.id, file=video_file, language="en")
  print(f"Created task: id={task.id}")

  # (Optional) Monitor the video indexing process
  # Utility function to print the status of a video indexing task
  def on_task_update(task: Task):
          print(f"  Status={task.status}")
  task.wait_for_done(sleep_interval=50, callback=on_task_update)
  if task.status != "ready":
      raise RuntimeError(f"Indexing failed with status {task.status}")
  print(f"Uploaded {video_file}. The unique identifer of your video is {task.video_id}.")

Uploading /Users/jaredyu/Desktop/projects/twelve_labs/technical_assignment/twelve-labs-sample-proj/static/videos/AWS AppFabric introduction.mp4
Created task: id=6670f2c92f5f583ac97c8cf8
  Status=pending
  Status=indexing
  Status=ready
Uploaded /Users/jaredyu/Desktop/projects/twelve_labs/technical_assignment/twelve-labs-sample-proj/static/videos/AWS AppFabric introduction.mp4. The unique identifer of your video is 6670f2c9d22b3a3c97bf2a74.


In [4]:
videos = client.index.video.list(index.id)
for video in videos:
    print(f"Generating text for {video.id}")

    res = client.generate.gist(video_id=video.id, types=["title", "topic", "hashtag"])
    print(f"Title: {res.title}\nTopics={res.topics}\nHashtags={res.hashtags}")

    res = client.generate.summarize(video_id=video.id, type="summary")
    print(f"Summary: {res.summary}")

    print("Chapters:")
    res = client.generate.summarize(video_id=video.id, type="chapter")
    for chapter in res.chapters:
        print(
            f"  chapter_number={chapter.chapter_number} chapter_title={chapter.chapter_title} chapter_summary={chapter.chapter_summary} start={chapter.start} end={chapter.end}"
        )

    print("Highlights:")
    res = client.generate.summarize(video_id=video.id, type="highlight")
    for highlight in res.highlights:
        print(
            f"  Highlight={highlight.highlight} start={highlight.start} end={highlight.end}"
        )

    res = client.generate.text(video_id=video.id, prompt="Based on this video, I want to generate five keywords for SEO (Search Engine Optimization).")
    print(f"Open-ended Text: {res.data}")

Generating text for 6670f2c9d22b3a3c97bf2a74
Title: AWS App Fabric: Seamlessly Connecting and Streamlining SaaS Applications for Enhanced Productivity and Security
Topics=[' AWS App Fabric - Connecting and Integrating SaaS Applications for Enhanced Productivity and Security.']
Hashtags=['AWS', 'cloud computing', 'SaaS', 'AppFabric', 'productivity', 'data management', 'generative AI']
Summary: The video serves as an informative and promotional piece for AWS (Amazon Web Services), focusing on the introduction of AWS App Fabric, a new solution designed to streamline and enhance the productivity and security of SaaS (Software as a Service) applications within organizations. It begins by highlighting the challenges faced by today's businesses, which typically utilize over 100 SaaS applications to boost employee productivity and satisfaction. Despite their benefits, these applications often operate in isolation, leading to data silos, security vulnerabilities, and inefficiencies as employees

In [7]:
res = client.generate.gist(video_id=video.id, types=["title", "topic", "hashtag"])

In [10]:
# video_name=video_summary.video_name,
# title=video_summary.title,
# topics=video_summary.topics,
# hashtags=video_summary.hashtags,
# summary=video_summary.summary,
# chapters=video_summary.chapters,
# highlights=video_summary.highlights,
# keywords=video_summary.keywords

# video_id=video_id,
# video_name=video.filename,
# title=res_gist.title,
# topics=res_gist.topics,
# hashtags=res_gist.hashtags,
# summary=res_summary.summary,
# chapters=res_chapters.chapters,
# highlights=res_highlights.highlights,
# keywords=res_keywords.data

In [11]:
res.title

'AWS App Fabric: Seamlessly Connecting and Streamlining SaaS Applications for Enhanced Productivity and Security'

In [12]:
res.topics # list of one string

[' AWS App Fabric - Connecting and Integrating SaaS Applications for Enhanced Productivity and Security.']

In [8]:
res.hashtags # list of strings

['AWS',
 'cloud computing',
 'SaaS',
 'AppFabric',
 'productivity',
 'data management',
 'generative AI']

In [13]:
res = client.generate.summarize(video_id=video.id, type="summary")

In [14]:
res.summary # long string

"The video provides a comprehensive overview of AWS App Fabric, a fully managed service designed to streamline the integration and management of SaaS applications within an organization. It begins by highlighting the challenges businesses face with the proliferation of purpose-built SaaS applications, noting that the average organization now utilizes over 100 SaaS applications. These applications, while boosting employee productivity and job satisfaction, often operate in silos due to being from different vendors. This not only creates data management and security headaches but also hampers employee efficiency due to the constant need to switch between different applications to access content or complete tasks.\nThe narrative then shifts to the traditional solution to this problem, which involves creating point-to-point integrations between each application. However, this approach is critiqued for being time-consuming, costly, and difficult to maintain. Here, AWS App Fabric is introduc

In [15]:
len(res.summary)

2470

In [27]:
res = client.generate.summarize(video_id=video.id, type="chapter")

In [30]:
import json

In [40]:
    # video_summary = VideoSummary.query.filter_by(video_id=video_id).first_or_404()
    # return render_template(
    #     'summary.html',
    #     video_name=video_summary.video_name,
    #     title=video_summary.title,
    #     topics=video_summary.topics,
    #     hashtags=video_summary.hashtags,
    #     summary=video_summary.summary,
    #     chapters=video_summary.chapters,
    #     highlights=video_summary.highlights,
    #     keywords=video_summary.keywords
    # )

In [ ]:
def process_bind_param(self, value, dialect):
    if value is not None:
        return json.dumps(value, default=lambda o: o.__dict__)
    return None

In [39]:
res.chapters

RootModelList[GenerateSummarizeChapterResult](root=[GenerateSummarizeChapterResult(chapter_number=0, start=0, end=102, chapter_title='Optimizing Productivity with AWS App Fabric', chapter_summary='The video introduces AWS App Fabric as a solution to the challenges of using multiple SaaS applications in an organization. It begins by highlighting the productivity benefits of purpose-built SaaS applications, noting that organizations typically use over 100 different SaaS applications, which can lead to data silos and integration challenges. AWS App Fabric is presented as a fully managed service that simplifies the connection of SaaS applications across an organization without the need for development, thereby enhancing IT and security management, and enabling employees to complete tasks more efficiently using generative AI. The service aims to eliminate the complexity of building and maintaining point-to-point integrations, improve security posture, and facilitate easier management of use

In [35]:
json.loads(json.dumps(res.chapters, default=lambda o: o.__dict__))

{'root': [{'chapter_number': 0,
   'start': 0,
   'end': 102,
   'chapter_title': 'Optimizing Productivity with AWS App Fabric',
   'chapter_summary': 'The video introduces AWS App Fabric as a solution to the challenges of using multiple SaaS applications in an organization. It begins by highlighting the productivity benefits of purpose-built SaaS applications, noting that organizations typically use over 100 different SaaS applications, which can lead to data silos and integration challenges. AWS App Fabric is presented as a fully managed service that simplifies the connection of SaaS applications across an organization without the need for development, thereby enhancing IT and security management, and enabling employees to complete tasks more efficiently using generative AI. The service aims to eliminate the complexity of building and maintaining point-to-point integrations, improve security posture, and facilitate easier management of user access and policies across applications. Th

In [ ]:
def process_result_value(self, value, dialect):
    if value is not None:
        return json.loads(value)
    return None

In [33]:
res.chapters

RootModelList[GenerateSummarizeChapterResult](root=[GenerateSummarizeChapterResult(chapter_number=0, start=0, end=102, chapter_title='Optimizing Productivity with AWS App Fabric', chapter_summary='The video introduces AWS App Fabric as a solution to the challenges of using multiple SaaS applications in an organization. It begins by highlighting the productivity benefits of purpose-built SaaS applications, noting that organizations typically use over 100 different SaaS applications, which can lead to data silos and integration challenges. AWS App Fabric is presented as a fully managed service that simplifies the connection of SaaS applications across an organization without the need for development, thereby enhancing IT and security management, and enabling employees to complete tasks more efficiently using generative AI. The service aims to eliminate the complexity of building and maintaining point-to-point integrations, improve security posture, and facilitate easier management of use

In [31]:
json.dumps(res.chapters)

TypeError: Object of type RootModelList[GenerateSummarizeChapterResult] is not JSON serializable

In [21]:
res.chapters[0]

GenerateSummarizeChapterResult(chapter_number=0, start=0, end=21, chapter_title='The Evolution of SaaS Applications', chapter_summary="The video begins by highlighting the increasing reliance on purpose-built SaaS applications to enhance team productivity and employee satisfaction. It notes the challenges posed by data silos created when applications from different vendors don't integrate seamlessly.")

In [22]:
res.chapters[1]

GenerateSummarizeChapterResult(chapter_number=1, start=22, end=42, chapter_title='Challenges of Application Integration', chapter_summary='The narrative shifts to the difficulties organizations face with data silos and the cumbersome nature of employees having to switch between applications. It also touches on the traditional approach of creating point-to-point integrations and its associated costs and maintenance challenges.')

In [23]:
res = client.generate.summarize(video_id=video.id, type="highlight")

In [24]:
res.highlights

RootModelList[GenerateSummarizeHighlightResult](root=[GenerateSummarizeHighlightResult(start=43, end=60, highlight='Introduction to AWS App Fabric'), GenerateSummarizeHighlightResult(start=61, end=82, highlight='Benefits of AWS App Fabric'), GenerateSummarizeHighlightResult(start=83, end=100, highlight='Enhancing Productivity with Generative AI'), GenerateSummarizeHighlightResult(start=1, end=42, highlight='Challenges of SaaS Applications')])

In [25]:
res = client.generate.text(video_id=video.id, prompt="Based on this video, I want to generate five keywords for SEO (Search Engine Optimization).")

In [26]:
res.data # one string

"Based on the comprehensive content of the video, which showcases the capabilities and services of AWS (Amazon Web Services), along with a detailed explanation of AWS App Fabric, the five most relevant keywords for SEO (Search Engine Optimization) would be: \n\n1. AWS App Fabric\n2. SaaS Applications Integration\n3. Generative AI\n4. Data Security Management\n5. Productivity Enhancement\n\nThese keywords encapsulate the core themes and technological solutions presented in the video, ranging from the integration of Software as a Service (SaaS) applications, leveraging generative AI for improving productivity, to managing and securing data across an organization's application ecosystem, all underpinned by AWS's infrastructure."

In [64]:
import os
from sqlalchemy import create_engine, MetaData, Table, select
from sqlalchemy.orm import sessionmaker
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.types import TypeDecorator, TEXT
import json

import os
from sqlalchemy import create_engine, MetaData, Table, Column, Integer, String, Text
from sqlalchemy.orm import sessionmaker
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.types import TypeDecorator
import json

In [65]:
# Define the database path (adjust this based on your actual database location)
db_path = os.path.join(os.getcwd(), 'database.db')

# Create SQLAlchemy engine
engine = create_engine(f'sqlite:///{db_path}')

# Create session factory
Session = sessionmaker(bind=engine)

# Create a session
session = Session()

In [66]:
class JSONType(TypeDecorator):
    impl = Text

    def process_bind_param(self, value, dialect):
        if value is not None:
            return json.dumps(value)
        return None

    def process_result_value(self, value, dialect):
        if value is not None:
            return json.loads(value)
        return None

In [67]:
# Create a base class for declarative class definitions
Base = declarative_base()

# Define your model class
class VideoSummary(Base):
    __tablename__ = 'video_summary'
    id = Column(Integer, primary_key=True)
    video_id = Column(String(200), nullable=False)
    video_name = Column(String(200), nullable=False)
    title = Column(String(500), nullable=False)
    topics = Column(JSONType, nullable=False)
    hashtags = Column(JSONType, nullable=False)
    summary = Column(Text, nullable=False)
    chapters = Column(JSONType, nullable=False)
    highlights = Column(JSONType, nullable=False)
    keywords = Column(JSONType, nullable=False)

# Reflect the tables
Base.metadata.reflect(engine)

/var/folders/23/4mlgy2qn2kd5mqznttb2pg2r0000gn/T/ipykernel_77946/3706886160.py:2: MovedIn20Warning: The ``declarative_base()`` function is now available as sqlalchemy.orm.declarative_base(). (deprecated since: 2.0) (Background on SQLAlchemy 2.0 at: https://sqlalche.me/e/b8d9)
  Base = declarative_base()


In [68]:
# Query all records from VideoSummary table
video_summaries = session.query(VideoSummary).all()

# Print the first video summary for inspection
if video_summaries:
    print(video_summaries[0].title, video_summaries[0].summary)
else:
    print("No video summaries found.")

OperationalError: (sqlite3.OperationalError) no such table: video_summary
[SQL: SELECT video_summary.id AS video_summary_id, video_summary.video_id AS video_summary_video_id, video_summary.video_name AS video_summary_video_name, video_summary.title AS video_summary_title, video_summary.topics AS video_summary_topics, video_summary.hashtags AS video_summary_hashtags, video_summary.summary AS video_summary_summary, video_summary.chapters AS video_summary_chapters, video_summary.highlights AS video_summary_highlights, video_summary.keywords AS video_summary_keywords 
FROM video_summary]
(Background on this error at: https://sqlalche.me/e/20/e3q8)

In [69]:
db_path = os.path.join(os.getcwd(), 'database.db')
db_path

'/Users/jaredyu/Desktop/projects/twelve_labs/technical_assignment/twelve-labs-sample-proj/database.db'

In [71]:
len("\
                    Please explain this music video. Explain the intention of the artist and their lyrics. \
                    Give cultural context so that the listener can know what angle the artist is coming from. \
                    What can we learn from the artist's or artists' words?")

291

In [58]:
video_summaries[0].chapters

{'root': [{'chapter_number': 0,
   'start': 0,
   'end': 21,
   'chapter_title': 'The Impact of SaaS Applications',
   'chapter_summary': "The video begins by highlighting the benefits of purpose-built SaaS applications in enhancing team productivity and employee satisfaction. It notes the average organization's use of over 100 SaaS applications but points out the challenge of applications from different vendors not being built to work together, creating data silos."},
  {'chapter_number': 1,
   'start': 22,
   'end': 42,
   'chapter_title': 'Challenges of SaaS Integration',
   'chapter_summary': 'The narrative shifts to the difficulties organizations face with data silos and the cumbersome nature of employees having to switch between applications. It discusses the attempts to solve this through point-to-point integrations, which are costly and time-consuming to develop and maintain.'},
  {'chapter_number': 2,
   'start': 43,
   'end': 60,
   'chapter_title': 'Introducing AWS App Fabri

In [59]:
video_summaries[0].highlights

{'root': [{'start': 43,
   'end': 60,
   'highlight': 'Introduction to AWS App Fabric'},
  {'start': 61, 'end': 82, 'highlight': 'Benefits of AWS App Fabric'},
  {'start': 83,
   'end': 100,
   'highlight': 'Enhancing Productivity with Generative AI'},
  {'start': 1, 'end': 42, 'highlight': 'Challenges with SaaS Applications'}]}

In [63]:
video_summaries[0].id

1

In [60]:
from filters import seconds_to_hhmmss

In [61]:

seconds_to_hhmmss

<function filters.seconds_to_hhmmss(seconds)>